# Does pricing affect usage?
We've all wondered if people really do use less energy when it costs more. Seems like it should. Let's test it just in case.

In [1]:
!pip install kedm

In [2]:
import kedm
import cudf
# from: https://www.nyiso.com/load-data
pricing_df = cudf.read_csv('http://mis.nyiso.com/public/csv/realtime/20210730realtime_zone.csv')
load_df = cudf.read_csv('http://mis.nyiso.com/public/csv/pal/20210730pal.csv')
pricing_df.to_csv('pricing.csv')
load_df.to_csv('load.csv')

pricing_and_load_df = pricing_df.merge(load_df, on=['Time Stamp', 'Name'], suffixes=['_pricing', '_load'])
pricing_and_load_df

,Time Stamp,Name,PTID_pricing,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr),Time Zone,PTID_load,Load
0,07/30/2021 09:00:00,HUD VL,61758,30.25,2.97,0.00,EDT,61758,1085.8429
1,07/30/2021 09:05:00,LONGIL,61762,53.63,1.74,-39.49,EDT,61762,2781.0193
2,07/30/2021 09:00:00,WEST,61752,48.84,0.33,-21.24,EDT,61752,1790.5573
3,07/30/2021 09:00:00,LONGIL,61762,45.78,3.74,-14.77,EDT,61762,2770.2869
4,07/30/2021 09:05:00,MHK VL,61756,15.18,0.64,-2.14,EDT,61756,897.8669
...,...,...,...,...,...,...,...,...,...
2173,07/30/2021 10:25:00,GENESE,61753,20.64,-0.06,-1.00,EDT,61753,1153.6170
2174,07/30/2021 10:30:00,HUD VL,61758,38.41,3.93,0.00,EDT,61758,1162.5972
2175,07/30/2021 10:25:00,HUD VL,61758,29.31,2.28,-7.32,EDT,61758,1172.3151
2176,07/30/2021 10:30:00,LONGIL,61762,45.87,5.21,-6.18,EDT,61762,2996.2656


Separate time series and turn into 2d table.

In [34]:
import cupy as cp
unique_names = pricing_and_load_df['Name'].unique()
selected_column_names = ['LBMP ($/MWHr)','Load']

input_timeseries = None
for i, name in enumerate(unique_names.to_array()):
    named_df = pricing_and_load_df[pricing_and_load_df['Name']==name]
    
    if input_timeseries is None:
        print(len(named_df))
        input_timeseries = cp.ndarray(((len(unique_names)) * (len(selected_column_names)),len(named_df)), dtype=cp.float64)
        input_timeseries.fill(0)
    col_df = named_df[selected_column_names]
    for j, col in enumerate(selected_column_names):
        col_ts = col_df[str(col)].to_gpu_array()
        col_ts = cp.resize(col_ts,len(named_df))
        input_timeseries[i + j] =col_ts
input_timeseries.get()

198


array([[44.29, 29.52, 35.22, ..., 31.93, 38.35, 30.27],
       [16.98, 28.45, 32.45, ..., 31.43, 35.93, 21.32],
       [38.66, 30.65, 35.96, ..., 34.35, 38.93, 29.67],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [219]:
import numpy as np

np_input = np.array(input_timeseries.get(),dtype=np.float32)[:12,:]
results = kedm.xmap(np.array(np_input.transpose()),[ kedm.edim(n) for n in np_input])
result_df = cudf.DataFrame()
for i, name in enumerate(unique_names.to_array()):
    if i*2+1 < len(results):
        result_df[str(name) + "_price"] = results[(i)*2]
        result_df[str(name) + "_load"] = results[(i)*2+1]
result_df.index=result_df.columns
import graphistry
graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="roshkins", password="Emptier6-Lard-Nativity-Mummy")
import pandas as pd
causal_graph = result_df.to_pandas()
causal_graph.values[[np.arange(len(causal_graph))]*2] = np.nan
causal_graph = causal_graph.stack().reset_index()
print(causal_graph.columns)
filtered_df = causal_graph[causal_graph[causal_graph.columns[2]] > 0.01]
graphistry.edges(filtered_df,'level_0','level_1').encode_edge_color('0', palette=['blue', 'red', 'yellow'], as_continuous=True).plot(memoize=False)


/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  from ipykernel import kernelapp as app


Index(['level_0', 'level_1', 0], dtype='object')


In [142]:
#!pip install graphistry

     |████████████████████████████████| 85 kB 6.2 MB/s  eta 0:00:01
